# Animal Classification 🦁

dataset : https://www.kaggle.com/datasets/antobenedetti/animals

## Importing Libraries

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

## Image Data Preprocessing

In [ ]:
target_size = (128, 128)

# creating an image data generator object and doing some data augmuntation on training set
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, #scaling each pixel to 0 to 1
                                                                shear_range=0.2,
                                                                zoom_range=0.2,
                                                                horizontal_flip=True)

# now connecting the object we just created above to train directory
train_generator = train_datagen.flow_from_directory('/kaggle/input/animals/animals/train', # traingset path directory
                                                    target_size=target_size, # images will become 128x128
                                                    batch_size=32,
                                                    class_mode='categorical') # because target is binary and not categorical

# creating another image data generator object, we dont use data augumnetation on val set!
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# connecting it to test directory
valid_generator = val_datagen.flow_from_directory('/kaggle/input/animals/animals/val',
                                                   target_size=target_size,
                                                   batch_size=32,
                                                   class_mode='categorical')

**Checking Data and labels**

In [ ]:
class_indices = train_generator.class_indices

# Print class indices and labels
for class_index, class_label in class_indices.items():
    print(f'Class Index: {class_index}, Class Label: {class_label}')

## Building CNN

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation=tf.keras.activations.relu, input_shape=(128, 128, 3)),
    tf.keras.layers.Conv2D(64, 3, activation=tf.keras.activations.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(64, 3, activation=tf.keras.activations.relu),
    tf.keras.layers.Conv2D(128, 3, activation=tf.keras.activations.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(128, 3, activation=tf.keras.activations.relu),
    tf.keras.layers.Conv2D(256, 3, activation=tf.keras.activations.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(units=256, activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(units=5, activation=tf.keras.activations.softmax)
])

## Compile the model

In [ ]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.003),
              metrics=['accuracy'])

## Training the model

In [ ]:
model.fit(train_generator,
          validation_data=valid_generator,
          epochs=30)

## Using our model to see how it performs

In [ ]:
image_path = '/kaggle/input/animals/animals/inf/elephant.jpg'

test_image = tf.keras.preprocessing.image.load_img(image_path, target_size=target_size)
test_array = tf.keras.preprocessing.image.img_to_array(test_image)
test_array = np.expand_dims(test_array, axis=0)

# Make prediction using our trained
result = model.predict(test_array)

# Create a reverse mapping from indices to labels
index_to_label = {index: label for label, index in class_indices.items()}

# Now, you can access the label using train_generator[result.argmax()]
print(f'Predicted Class Label: {index_to_label[np.argmax(result)]}')

# Check if the predicted class is the one you're interested in
predicted_class_index = np.argmax(result)

if predicted_class_index == 2: # 2 because elephent index is 2
    color='green'
else:
    color='red'

# Visualize the image with the prediction as the title
plt.imshow(test_image)
plt.title(f'Prediction: {index_to_label[np.argmax(result)]}', color=color, fontsize=14, fontweight='bold')
plt.axis('off')
plt.show()